In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import stats
import glob
from scipy.stats import ks_2samp, kstest
%matplotlib inline

In [2]:
def load_summary(filename):
    dtype=[('minr', 'f8'),
           ('maxr', 'f8'), 
           ('ca_ratio', 'f8'),
           ('ba_ratio', 'f8'),
           ('a', 'f8'),
           ('center', 'f8'),
           ('width', 'f8'),
           ('mu', 'f8')]
    summary = np.loadtxt(filename, dtype=dtype)    
    return summary

In [3]:
def load_experiment(input_path="../data/mstar_selected_summary/vmax_sorted/", fixed_number=False, full_data=False):
    files = glob.glob(input_path+"M31_group_*")
    group_id = []
    for f in files:
        i = int(f.split("_")[-5])
        if i not in group_id:
            group_id.append(i)
    print(group_id, len(group_id))

    n_groups = len(group_id)

    if fixed_number:
        n_iter = np.arange(5)
    else:
        n_iter = np.arange(11,16)
    
    fields = ['width','mu', 'a', 'ba_ratio', 'ca_ratio']
    M31_all = {}
    MW_all = {}
    if not full_data:
        for field in fields:
            M31_all[field] = np.ones(n_groups)
            MW_all[field] = np.ones(n_groups)
            M31_all[field+'_sigma'] = np.ones(n_groups)
            MW_all[field+'_sigma'] = np.ones(n_groups)
        
            M31_all[field+'_random'] = np.ones(n_groups)
            MW_all[field+'_random'] = np.ones(n_groups)
            M31_all[field+'_random_sigma'] = np.ones(n_groups)
            MW_all[field+'_random_sigma'] = np.ones(n_groups)
    else:
        for field in fields:
            M31_all[field] = np.empty((0))
            MW_all[field] = np.empty((0))
            M31_all[field+'_random'] = np.empty((0))
            MW_all[field+'_random'] = np.empty((0))
           

    for g in range(n_groups):

        MW_summary = {}
        M31_summary = {}
    
        for i in n_iter:
            if fixed_number:
                filename_MW = os.path.join(input_path,"MW_group_{}_nmax_{}_iter_{}.dat".format(group_id[g], 11, i))
                filename_M31 = os.path.join(input_path,"M31_group_{}_nmax_{}_iter_{}.dat".format(group_id[g],11, i))
            else:
                filename_MW = os.path.join(input_path,"MW_group_{}_nmax_{}_iter_{}.dat".format(group_id[g], i, 0))
                filename_M31 = os.path.join(input_path,"M31_group_{}_nmax_{}_iter_{}.dat".format(group_id[g], i, 0))

            MW_summary[i] = load_summary(filename_MW)
            M31_summary[i] = load_summary(filename_M31)
    
        
        for field in fields:
            a = np.empty((0))
            b = np.empty((0))
            a_random = np.empty((0))
            b_random = np.empty((0))
        
            for i in n_iter:
                data = M31_summary[i]
                a = np.append(a, data[field][0])
                a_random = np.append(a_random, data[field][1:101])
        
                data = MW_summary[i]
                b = np.append(b, data[field][0])
                b_random = np.append(b_random, data[field][1:101])
                #print('a_random {} iter: {} {}'.format(field, i, a_random))
                
            if not full_data:
                M31_all[field][g] = np.average(a)
                MW_all[field][g] = np.average(b)
                M31_all[field+'_sigma'][g] = np.std(a)
                MW_all[field+'_sigma'][g] = np.std(b)
                M31_all[field+'_random'][g] = np.average(a_random)
                MW_all[field+'_random'][g] = np.average(b_random)
                M31_all[field+'_random_sigma'][g] = np.std(a_random)
                MW_all[field+'_random_sigma'][g] = np.std(b_random)
            else:
                M31_all[field] = np.append(M31_all[field], a)
                MW_all[field] = np.append(MW_all[field], b)
                M31_all[field+'_random'] = np.append(M31_all[field+'_random'], a_random)
                MW_all[field+'_random'] = np.append(MW_all[field+'_random'], b_random)
                
    return M31_all, MW_all

In [4]:
in_path = "../data/obs_summary/vmag_sorted/"
M31_obs_vmag_sorted, MW_obs_vmag_sorted = load_experiment(input_path=in_path, fixed_number=False, full_data=False)

in_path = "../data/mstar_selected_summary/vmax_sorted/"
M31_sim_vmax_sorted, MW_sim_vmax_sorted = load_experiment(input_path=in_path, fixed_number=False)

in_path = "../data/elvis_mstar_selected_summary/vmax_sorted/"
M31_sim_vmax_sorted_elvis, MW_sim_vmax_sorted_elvis = load_experiment(input_path=in_path, fixed_number=False)

[0] 1
[0, 10, 11, 13, 14, 16, 18, 1, 20, 21, 22, 24, 25, 2, 3, 4, 5, 6, 8, 9] 20
[0, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 9] 12


In [5]:
print("M31 observations \n")
fields = ['width', 'ca_ratio', 'ba_ratio']
for field in fields:
    print("Natural units\n", field, M31_obs_vmag_sorted[field][0], M31_obs_vmag_sorted[field+'_sigma'][0])
    
    
print("\nMW observations \n")
fields = ['width', 'ca_ratio', 'ba_ratio']
for field in fields:
    print("Natural units\n", field, MW_obs_vmag_sorted[field][0], MW_obs_vmag_sorted[field+'_sigma'][0])

M31 observations 

Natural units
 width 59.06998 3.00637879742
Natural units
 ca_ratio 0.4514 0.0409125897494
Natural units
 ba_ratio 0.8226 0.0680370487308

MW observations 

Natural units
 width 22.78204 2.06005897061
Natural units
 ca_ratio 0.2846 0.0375638123731
Natural units
 ba_ratio 0.7828 0.025926048677


In [6]:
print("M31 observations \n")
fields = ['width', 'ca_ratio', 'ba_ratio']
for field in fields:    
    normed_mean = (M31_obs_vmag_sorted[field][0] - M31_obs_vmag_sorted[field+'_random'][0])/M31_obs_vmag_sorted[field+'_random_sigma'][0]
    normed_sigma = M31_obs_vmag_sorted[field+'_sigma'][0]/M31_obs_vmag_sorted[field+'_random_sigma'][0]

    print("Normalized units\n", field, normed_mean, normed_sigma)

    
print("\nMW observations \n")
fields = ['width', 'ca_ratio', 'ba_ratio']
for field in fields:
    
    normed_mean = (MW_obs_vmag_sorted[field][0] - MW_obs_vmag_sorted[field+'_random'][0])/MW_obs_vmag_sorted[field+'_random_sigma'][0]
    normed_sigma = MW_obs_vmag_sorted[field+'_sigma'][0]/MW_obs_vmag_sorted[field+'_random_sigma'][0]

    print("Normalized units\n", field, normed_mean, normed_sigma)

M31 observations 

Normalized units
 width -0.475222114569 0.248398557688
Normalized units
 ca_ratio -1.00436515793 0.379861886083
Normalized units
 ba_ratio -0.00265352060199 0.851592974084

MW observations 

Normalized units
 width -2.73434559175 0.245348995315
Normalized units
 ca_ratio -2.40984663643 0.356151500566
Normalized units
 ba_ratio -0.348831312222 0.312718450232


In [7]:
print("M31 observations (spherically randomized)\n")
fields = ['width', 'ca_ratio', 'ba_ratio']
for field in fields:
    print("Natural units\n", field, M31_obs_vmag_sorted[field+'_random'][0], M31_obs_vmag_sorted[field+'_random_sigma'][0])
    
print("\nMW observations (spherically randomized)\n")
fields = ['width', 'ca_ratio', 'ba_ratio']
for field in fields:
    print("Natural units\n", field, MW_obs_vmag_sorted[field+'_random'][0], MW_obs_vmag_sorted[field+'_random_sigma'][0])


M31 observations (spherically randomized)

Natural units
 width 64.8216144 12.1030445
Natural units
 ca_ratio 0.559574 0.10770385566
Natural units
 ba_ratio 0.822812 0.0798938586876

MW observations (spherically randomized)

Natural units
 width 45.7408182 8.39644347418
Natural units
 ca_ratio 0.53877 0.105471442106
Natural units
 ba_ratio 0.81172 0.0829054015128


In [8]:
print("M31 illustris simulation \n")
fields = ['width', 'ca_ratio', 'ba_ratio']
for field in fields:
    print("Natural units\n", field, 
          np.mean(M31_sim_vmax_sorted[field]), np.std(M31_sim_vmax_sorted[field+'_sigma']))
    
    
print("\nMW illustris simulation \n")
fields = ['width', 'ca_ratio', 'ba_ratio']
for field in fields:
    print("Natural units\n", field, 
          np.mean(MW_sim_vmax_sorted[field]), np.std(MW_sim_vmax_sorted[field+'_sigma']))
  

M31 illustris simulation 

Natural units
 width 70.679036 4.4508046938
Natural units
 ca_ratio 0.52898 0.0185844310273
Natural units
 ba_ratio 0.80847 0.0196257712254

MW illustris simulation 

Natural units
 width 67.187621 2.09660462277
Natural units
 ca_ratio 0.5311 0.0125148520275
Natural units
 ba_ratio 0.80566 0.0200189978772


In [9]:
print("M31 elvis simulation \n")
fields = ['width', 'ca_ratio', 'ba_ratio']
for field in fields:
    print("Natural units\n", field, 
          np.mean(M31_sim_vmax_sorted_elvis[field]), np.std(M31_sim_vmax_sorted_elvis[field+'_sigma']))
    
    
print("\nMW elvis simulation \n")
fields = ['width', 'ca_ratio', 'ba_ratio']
for field in fields:
    print("Natural units\n", field, 
          np.mean(MW_sim_vmax_sorted_elvis[field]), np.std(MW_sim_vmax_sorted_elvis[field+'_sigma']))
  

M31 elvis simulation 

Natural units
 width 70.61085 1.96390149498
Natural units
 ca_ratio 0.54725 0.017208460608
Natural units
 ba_ratio 0.80925 0.019973160614

MW elvis simulation 

Natural units
 width 68.82925 4.17411639232
Natural units
 ca_ratio 0.490216666667 0.0246489824716
Natural units
 ba_ratio 0.818166666667 0.0190564260027
